# Simple regression

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mllib').getOrCreate()

In [ ]:
spark

In [6]:
## read dataset
train = spark.read.csv('mllib_test.csv', header = True, inferSchema = True, sep = ';')

In [7]:
train.show()

+-----+---+---+------+
| Name|Age|Exp|Salary|
+-----+---+---+------+
|hinat| 31| 10| 30000|
| suki| 30|  8| 25000|
| momo| 29|  4| 20000|
|  zum| 24|  3| 20000|
| maki| 21|  1| 15000|
|  avo| 23|  2| 18000|
+-----+---+---+------+



In [8]:
train.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Exp: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
train.columns

['Name', 'Age', 'Exp', 'Salary']

In [10]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['Age', 'Exp'], outputCol = 'Independent features')

In [12]:
output = featureassembler.transform(train)

In [13]:
output.show()

+-----+---+---+------+--------------------+
| Name|Age|Exp|Salary|Independent features|
+-----+---+---+------+--------------------+
|hinat| 31| 10| 30000|         [31.0,10.0]|
| suki| 30|  8| 25000|          [30.0,8.0]|
| momo| 29|  4| 20000|          [29.0,4.0]|
|  zum| 24|  3| 20000|          [24.0,3.0]|
| maki| 21|  1| 15000|          [21.0,1.0]|
|  avo| 23|  2| 18000|          [23.0,2.0]|
+-----+---+---+------+--------------------+



In [14]:
output.columns

['Name', 'Age', 'Exp', 'Salary', 'Independent features']

In [15]:
finalized_data = output.select('Independent features', 'Salary')

In [16]:
finalized_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [26]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'Independent features', labelCol = 'Salary')
regressor = regressor.fit(train_data)

In [27]:
### coef
regressor.coefficients

DenseVector([2666.6667, -2000.0])

In [28]:
### intercepts
regressor.intercept

-38833.33333337073

In [29]:
### prediction
pred_results = regressor.evaluate(test_data)

In [30]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent features|Salary|        prediction|
+--------------------+------+------------------+
|          [29.0,4.0]| 20000| 30500.00000000719|
|         [31.0,10.0]| 30000|23833.333333330404|
+--------------------+------+------------------+



In [31]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(8333.333333338393, 74138888.88898243)

In [ ]:
# from pyspark.ml.feature import StringIndexer

In [32]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x000001D44FD84FD0>>